# pyiron

## Define workflow with pyiron_base

In [1]:
from pyiron_base import job

In [2]:
from python_workflow_definition.pyiron_base import write_workflow_json

In [ ]:
from workflow import (
    get_sum as _get_sum,
    get_prod_and_div as _get_prod_and_div,
    get_square as _get_square,
)

In [ ]:
workflow_json_filename = "pyiron_arithmetic.json"

In [ ]:
get_sum = job(_get_sum)
get_prod_and_div = job(_get_prod_and_div, output_key_lst=["prod", "div"])
get_square = job(_get_square)

In [ ]:
prod_and_div = get_prod_and_div(x=1, y=2)

In [ ]:
tmp_sum = get_sum(x=prod_and_div.output.prod, y=prod_and_div.output.div)

In [ ]:
result = get_sum(x=tmp_sum.output)

In [ ]:
write_workflow_json(delayed_object=result, file_name=workflow_json_filename)

In [9]:
!cat {workflow_json_filename}

{"nodes": [{"id": 0, "function": "arithmetic_workflow.get_sum"}, {"id": 1, "function": "arithmetic_workflow.get_prod_and_div"}, {"id": 2, "value": 1}, {"id": 3, "value": 2}], "edges": [{"target": 0, "targetPort": "x", "source": 1, "sourcePort": "prod"}, {"target": 1, "targetPort": "x", "source": 2, "sourcePort": null}, {"target": 1, "targetPort": "y", "source": 3, "sourcePort": null}, {"target": 0, "targetPort": "y", "source": 1, "sourcePort": "div"}]}

## Load Workflow with aiida

In [10]:
from aiida import load_profile

load_profile()

Profile<uuid='5b8a8d98b3014876a9eed7d85a954e5e' name='pwd'>

In [11]:
from python_workflow_definition.aiida import load_workflow_json

In [12]:
wg = load_workflow_json(file_name=workflow_json_filename)
wg

NodeGraphWidget(settings={'minimap': True}, style={'width': '90%', 'height': '600px'}, value={'name': 'WorkGra…

In [13]:

wg.run()

04/23/2025 09:45:47 AM <4032> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [138|WorkGraphEngine|continue_workgraph]: tasks ready to run: get_prod_and_div2
04/23/2025 09:45:48 AM <4032> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [138|WorkGraphEngine|update_task_state]: Task: get_prod_and_div2, type: PyFunction, finished.
04/23/2025 09:45:48 AM <4032> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [138|WorkGraphEngine|continue_workgraph]: tasks ready to run: get_sum1
04/23/2025 09:45:48 AM <4032> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [138|WorkGraphEngine|update_task_state]: Task: get_sum1, type: PyFunction, finished.
04/23/2025 09:45:48 AM <4032> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [138|WorkGraphEngine|continue_workgraph]: tasks ready to run: 
04/23/2025 09:45:48 AM <4032> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [138|WorkGraphEngine|fina

## Load Workflow with jobflow

In [14]:
from python_workflow_definition.jobflow import load_workflow_json

/srv/conda/envs/notebook/lib/python3.12/site-packages/paramiko/pkey.py:82: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 48.0.0.
  "cipher": algorithms.TripleDES,
/srv/conda/envs/notebook/lib/python3.12/site-packages/paramiko/transport.py:253: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 48.0.0.
  "class": algorithms.TripleDES,


In [15]:
from jobflow.managers.local import run_locally

In [16]:
flow = load_workflow_json(file_name=workflow_json_filename)

In [17]:
result = run_locally(flow)
result

2025-04-23 09:45:49,174 INFO Started executing jobs locally
2025-04-23 09:45:49,540 INFO Starting job - get_prod_and_div (77d76447-21f8-4ad8-b2b5-bef7263ef5b9)
2025-04-23 09:45:49,543 INFO Finished job - get_prod_and_div (77d76447-21f8-4ad8-b2b5-bef7263ef5b9)
2025-04-23 09:45:49,544 INFO Starting job - get_sum (e61e0bc0-bddd-4ac9-93f0-cf5f064e17b0)
2025-04-23 09:45:49,546 INFO Finished job - get_sum (e61e0bc0-bddd-4ac9-93f0-cf5f064e17b0)
2025-04-23 09:45:49,546 INFO Finished executing jobs locally


{'77d76447-21f8-4ad8-b2b5-bef7263ef5b9': {1: Response(output={'prod': 2, 'div': 0.5}, detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False, job_dir=PosixPath('/home/jovyan'))},
 'e61e0bc0-bddd-4ac9-93f0-cf5f064e17b0': {1: Response(output=2.5, detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False, job_dir=PosixPath('/home/jovyan'))}}